In [1]:
# Amaris Efthimiou
# Weiwei Gao
# Weisheng Aw5
# Cali Stanford

In [2]:
import pandas as pd
import numpy as np
import ast
from ast import literal_eval
from pprint import PrettyPrinter
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

In [3]:
! ls -l 'tmdb_5000_movies.csv' 'tmdb_5000_credits.csv'

-rw-r--r--  1 amarisefthimiou  staff  40044293 Apr  1 17:18 tmdb_5000_credits.csv
-rw-r--r--  1 amarisefthimiou  staff   5698602 Apr  1 17:18 tmdb_5000_movies.csv


In [4]:
! wc -l 'tmdb_5000_movies.csv' 'tmdb_5000_credits.csv'

    4804 tmdb_5000_movies.csv
    4804 tmdb_5000_credits.csv
    9608 total


In [5]:
#! head 'tmdb_5000_movies.csv'

In [6]:
#! head 'tmdb_5000_credits.csv'

In [7]:
# *** PROJECT DESCRIPTION ***
# All of these attributes will help return relevant movie titles on the basis of 
# cast members, spoken language, overview (plot), and genre. 
# We chose these features because when we look for movies to watch in our everyday lives, we agree that those 
# 4 attributes are the biggest determinators of whether or not we decide to watch. For example, if you want a 
# movie similar to the Titanic, you would want the recommendations to offer movies in a romance genre, movies with 
# Leonardo DiCaprio, movies in English, and movies similar to Titanic’s plot.

In [8]:
# Reading data files
movies_df = pd.read_csv('tmdb_5000_movies.csv')
credits_df = pd.read_csv('tmdb_5000_credits.csv')

In [9]:
credits_df.columns = ['id','title','cast','crew']
movies_df= movies_df.merge(credits_df,on='id')

In [10]:
print(movies_df)

         budget                                             genres  \
0     237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1     300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2     245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3     250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4     260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
...         ...                                                ...   
4798     220000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4799       9000  [{"id": 35, "name": "Comedy"}, {"id": 10749, "...   
4800          0  [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...   
4801          0                                                 []   
4802          0                [{"id": 99, "name": "Documentary"}]   

                                               homepage      id  \
0                           http://www.avatarmovie.com/   19995   
1          http://disney.

In [11]:
# Clean-up data: remove newline characters in 'overview'
#movies_df['overview'] = movies_df['overview'].str.replace(r'\n', '', regex=True)
#movies_df.sample(10)

In [12]:
# was getting "float" error; not able to iterate, so chnaged type to string
movies_df['overview'] = movies_df['overview'].astype(str)

In [13]:
#fucntion to remove punctuation from 'overview'
string.punctuation
def remove_punct(text):
    text_nopunct = "".join([char for char in text if char not in string.punctuation])
    return text_nopunct

# Remove punctuation from data
movies_df['overview'] = movies_df['overview'].apply(lambda x: remove_punct(x))
movies_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,In the 22nd century a paraplegic Marine is dis...,150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,Captain Barbossa long believed to be dead has ...,139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,John Carter is a warweary former military capt...,43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [14]:
# Calculate the TF-IDF score for each word in overview, word by word
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')#removing stop words like 'the' and 'a'

# overview_matrix variable where we store the matrix containing each word and 
#its TF-IDF score with regard to each word in overview
overview_matrix = tfidf.fit_transform(movies_df['overview'])
overview_matrix

<4803x23203 sparse matrix of type '<class 'numpy.float64'>'
	with 125199 stored elements in Compressed Sparse Row format>

In [15]:
#finding cosine similarity to measure similarity of plots/'overview' for each movie
cosine_sim2 = linear_kernel(overview_matrix, overview_matrix)

In [16]:
# next set of features to measure --> spoken_languages, keywords, genres, and cast
features = ['spoken_languages', 'keywords', 'genres', 'cast']

In [17]:
#converting string list features in tuples
for feature in features: movies_df[feature] = movies_df[feature].apply(lambda x: literal_eval(str(x)))

In [18]:
#function that only takes the first 2 names from the list 
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 2:
            names = names[:2]
        return names

In [19]:
#using the get_list function to only return the first two names in the list of features 
#and iterate that for every feature
for feature in features:
   movies_df[feature] = movies_df[feature].apply(get_list) #turn into list

In [20]:
movies_df[['original_title','keywords', 'genres', 'spoken_languages', 'cast']].head(5)
#these are the CLEANED features we are going to work with (excluding original_title)

,original_title,keywords,genres,spoken_languages,cast
0,Avatar,"[culture clash, future]","[Action, Adventure]","[English, Español]","[Sam Worthington, Zoe Saldana]"
1,Pirates of the Caribbean: At World's End,"[ocean, drug abuse]","[Adventure, Fantasy]",[English],"[Johnny Depp, Orlando Bloom]"
2,Spectre,"[spy, based on novel]","[Action, Adventure]","[Français, English]","[Daniel Craig, Christoph Waltz]"
3,The Dark Knight Rises,"[dc comics, crime fighter]","[Action, Crime]",[English],"[Christian Bale, Michael Caine]"
4,John Carter,"[based on novel, mars]","[Action, Adventure]",[English],"[Taylor Kitsch, Lynn Collins]"


In [21]:
# reference --> O'Reilly "Hands on Reccomendation Systems"
#creating a soup out of the metadata to join the combined features in a single string, in order to feed it into
#the vectorizer
def rec_features(x):
    return ' '.join(x['keywords']) + '  '.join(x['genres']) + '  '.join(x['spoken_languages']) + '  '.join(x['cast'])# + ' ' + ' '.join(x['overview'])
movies_df['rec_features'] = movies_df.apply(rec_features, axis=1)

In [22]:
#print(movies_df)

In [23]:
countVec = CountVectorizer(analyzer='word', stop_words='english') #removing unnecessary stop words
#in order to extract features from our data, must use fit.transform to 
#count the number of texts and make the transformed matrix (keyword_matrix) into an array
keyword_matrix = countVec.fit_transform(movies_df['rec_features'])

In [24]:
cosine_sim = cosine_similarity(keyword_matrix, keyword_matrix) 
# cosine_sim measures distance between similarities of movie features chosen

In [25]:
# Reset index of main DataFrame and construct reverse mapping
movies_df = movies_df.reset_index()
movies = pd.Series(movies_df.index, index=movies_df['original_title'])
#we conduct reverse mapping to identify the index of a movie by the original_title

In [26]:
cosine_sim3 = cosine_similarity(cosine_sim, cosine_sim2)
#decided to take the cosine similarity of the cosine similiarity score of the 'overview' 
#dataframe coloumn and the similarity score from the features, in order to measure the similarities
#between movies using overview AND other features.
#why did we do it like this ? --> we were not able to use the 'overview' column in our first set of features;
#it is a float object with many features and the data can't be cleaned together with the rest of the features.
#so, we decided to measure its similarity scores seperately, then finally combine the scores 
#to create a more powerful scorer.

In [27]:
def get_recommendations(title, cosine_sim3=cosine_sim3):
    # index of the movie with that title
    recommendation = movies[title]
    # pairwise similarity scores of all movies with that movie name
    similarity = list(enumerate(cosine_sim3[recommendation]))
    # sorting the movies based on similarity score
    similarity = sorted(similarity, key=lambda x: x[1], reverse=True)
    # gets scores of 5 most similar movies
    similarity = similarity[1:6]
    # gets the index of the similar movies
    movie_recommendation = [i[0] for i in similarity]
    # return top 5 most similar movies
    return movies_df['original_title'].iloc[movie_recommendation]

In [28]:
movie = input('Enter a movie for recommendations: ')

Enter a movie for recommendations: 21 Jump Street


In [29]:
get_recommendations(movie, cosine_sim)

911      22 Jump Street
2660     Magic Mike XXL
1879         The Sitter
928           Moneyball
1500    This Is the End
Name: original_title, dtype: object

In [30]:
#REFERENCES --> all of the sources that helped us along the way!
# C8_2_TM_2 JupyterNotebook
# C9_2_RS_2 JupyterNotebook
#https://towardsdatascience.com/mixed-recommender-system-mf-matrix-factorization-with-item-similarity-based-cf-collaborative-544ddcedb330
#https://towardsdatascience.com/using-cosine-similarity-to-build-a-movie-recommendation-system-ae7f20842599
#https://www.datacamp.com/community/tutorials/recommender-systems-python
#https://www.oreilly.com/library/view/hands-on-recommendation-systems/9781788993753/0cf5a06a-8e47-4634-ac17-9bf2d5431f40.xhtml
#https://towardsdatascience.com/using-cosine-similarity-to-build-a-movie-recommendation-system-ae7f20842599